In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch2.Ollama_LLM활용의기본개념(LangChain)</span>

# 1. LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM이용
성능은 GPT, Claude 같은 모델보다는 떨어지나, 개념 설명을 위해 open source 모델 사용

### ollama.com 다운로드 - > 설치 -> 모델 pull
- cmd창이나 powershell 창에 ollama run deepseek-r1:1.5b
- https://docs.langchain.com/oss/python/integrations/chat/ollama

In [3]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = "deepseek-r1:1.5b")
result = llm.invoke("What is the capital of Korea?")
result

ConnectError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

### 모델 pull

- cmd 창이나 powershell창(window키+R에서 powershell)에서 ollama pull llama3.2:1b

In [4]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of Korea?")
result

ConnectError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

In [ ]:
result.content

In [5]:
result = llm.invoke("한국 수도가 어디예요?")
result.content

ConnectError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

## 2) openai 활용
- pip install langchain-openai
- https://auth.openai.com/log-in

In [6]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# os.getenv("OPENAI_API_KEY")

True

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano",
                #api_key=os.getenv("OPENAI_API_KEY")
                )
result = llm.invoke("What is the capital of Korea?  Return the name of the city only.")

In [8]:
result

AIMessage(content='Seoul', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 331, 'prompt_tokens': 22, 'total_tokens': 353, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CkliSgXz4edRIzJOBzDJVCu65hxlB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b01d8-c86a-7d51-987f-ea175c3052ec-0', usage_metadata={'input_tokens': 22, 'output_tokens': 331, 'total_tokens': 353, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 320}})

In [ ]:
# Azure : OPENAI_API_VERSION키값
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model="gpt-5-nano")

# 2. 렝체인 스타일로 프롬프트 작성

- 프롬프트 : llm 호출시 쓰는 질문

In [9]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
#result = llm.invoke(55)
# PromptValue, str, BaseMessages리스트

## 1) 기본 프롬프트 템플릿 사용

- PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [10]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)
country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":country})
print(prompt)
llm.invoke(prompt)

어느 나라의 수도를 알고 싶으신가요?korea
text='What is the capital of korea?'


AIMessage(content='The capital of Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:42:45.4423545Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3110527300, 'load_duration': 2518836800, 'prompt_eval_count': 33, 'prompt_eval_duration': 308639500, 'eval_count': 8, 'eval_duration': 271182600, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d9-141a-7a11-bb8e-67b4e63345d6-0', usage_metadata={'input_tokens': 33, 'output_tokens': 8, 'total_tokens': 41})

## 2) 메세지 기반 프롬프트 작성

- BaseMessage 리스트
- BaseMessage 상속받은 클래스 :  AIMessage, HumanMessage, SystemMessage, ToolMessage
- vscode에서 ctrl+shift+p : python:select interpreter입력 -> python환경선택
- vscode에서 커널 선택

In [11]:
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of korea?")
]

In [12]:
llm.invoke(message_list)

AIMessage(content='The capital of Korea (specifically, South Korea) is Seoul. The capital of North Korea is Pyongyang.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:42:50.6357263Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1656867600, 'load_duration': 124549900, 'prompt_eval_count': 87, 'prompt_eval_duration': 665303900, 'eval_count': 23, 'eval_duration': 847012100, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d9-2e11-7b60-8245-d36f53a79dff-0', usage_metadata={'input_tokens': 87, 'output_tokens': 23, 'total_tokens': 110})

In [13]:
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of {country}?")
]
print(message_list)

[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Rome.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of {country}?', additional_kwargs={}, response_metadata={})]


## 3) ChatPromptTemplate 사용

- BaseMessage 리스트 : 튜플리스트

In [14]:
# 위의 BaseMessage리스트를 수정
# PromptTemplate : 프롬프트에 변수포함, 
# ChatPromptTemplate : SystemPromt설정(페르소나), few shot 설정, 변수포함
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
ChatPrompt_template=ChatPromptTemplate.from_messages([
    ("system","You are  a helpfull assistant!"),
    ("human","What is the capital of Italy?"),
    ("ai","The capital of Italy is Rome."),
    ("human","What is the capital of France?"),
    ("ai","The capital of France is Paris."),
    ("human","What is the capital of {country}?")
])
country = input(" 어느 나라 수도가 궁금하세요??")
prompt = ChatPrompt_template.invoke({"country":country})
# print("프롬프트 : ", prompt, type(prompt))
result = llm.invoke(prompt)
result.content

 어느 나라 수도가 궁금하세요??korea


'The capital of Korea is Seoul.'

In [15]:
chatPromptTemplate = ChatPromptTemplate.from_messages([
    ("system", "당신은 대한민국 정보 전문 도우미입니다"),
    ("human", "{country}의 수도가 어디예요!")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPromptTemplate.invoke({"country":country})
# print(prompt)
result = llm.invoke(prompt)
print(result.content)

어느 나라 수도가 궁금하세요korea
 korea's 수도는 Seoul입니다.


# 3. 답변 형식 컨트롤하기

- llm.invoke()의 결과는 AIMessage() -> string이나 json, 객체 : OutputParser 이용

## 1) 문자열 출력 파서 이용
- StrOutputParser를 사용하여 LLM출력( AIMessage)을 단순 문자열로 변환

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Return the name of the city only.",
    input_variables = ["country"]
)
# 프롬프트 템플릿에 값을 주입
prompt = prompt_template.invoke({"country":"korea"})
# print(prompt)
ai_message = llm.invoke(prompt)
# print(ai_message)

# 문자열 출력 파서를 이용하여 llm응답(AIMessage객체)을 단순 문자열로 변환
output_parser =  StrOutputParser()
result = output_parser.invoke(ai_message)
result

'Seoul'

In [23]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

In [25]:
# 변수설정, system, few shot 지정
chat_prompt_template = ChatPromptTemplate([
    ("system","You are a helpfull assistant with expertise in South Korea."),
    ("human","What is the capital of Italy?"), # 모범질문
    ("ai","Rome."), # 모범답안
    ("human","What is the capital of France?"), # 모범질문
    ("ai","Paris."), # 모범답안
    ("human","What is the capital of {country}? Return the name of the city only") # Return the name of the city only
])
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) Json 출력파서 이용

- {'name':'홍길동','age':22}

In [39]:
from langchain_core.output_parsers import JsonOutputParser
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
prompt = country_detail_prompt.invoke({"country":"Korea"})
# print(prompt)
ai_message = llm.invoke(prompt)
# print(ai_message.content)
output_parser = JsonOutputParser()
json_result = output_parser.invoke(ai_message)
print(json_result,type(json_result))

{'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'KRW'} <class 'dict'>


In [38]:
output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country":"Japan"})))

{'capital': 'Tokyo',
 'population': '128,000,000',
 'language': 'Japanese',
 'currency': 'Japanese Yen'}

## 3) 구조화된 출력 사용

- Pydantic 모델을 사용하여 LLM출력을 구조화된 형식으로 받기(JsonParser보다 훨씬 안정적)
- Pydantic : 데이터유효성검사, 설정 관리를 간편하게 해주는 라이브러리

In [48]:
class User:
    def __init__(self,id,name,is_active=True):
        self.id = id
        self.name = name
        self.is_active = is_active
#     def __str__(self):
#         return self.name
user = User(1,"홍길동")
print(user)

In [53]:
from pydantic import BaseModel, Field
class User(BaseModel):
    # gt=0:id>0 /ge=0:id>=0 / lt=0:id<0 /le=0:id<=0
    id:int = Field(gt=0,description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id활셩화 여부")
user = User(id=1,name="홍길동")
print(user)
user = User(id="1",name="홍길동")
print(user)
# user = User(id="일",name="홍길동")
# print(user)

id=1 name='홍길동' is_active=True
id=1 name='홍길동' is_active=True


ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='일', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

In [56]:
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
class CountryDetail(BaseModel): # description : 더 정확한 출력 유도
    capital:str = Field(description="the Capital of the country")
    population:int = Field(description="the Capital of the country")
    language:str = Field(description="the language of the country")
    currency:str = Field("the currency of the country")
# 출력 형식 파서 + LLM
structedllm = llm.with_structured_output(CountryDetail)
#llm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info = structedllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))            
info

CountryDetail(capital='Seoul', population=51, language='Korean', currency='South Korean won')

In [57]:
type(info)

__main__.CountryDetail

In [58]:
info.capital, info.population, info.language,info.currency

('Seoul', 51, 'Korean', 'South Korean won')

In [63]:
print("info를 json으로 : ", info.model_dump_json())
print("info를 dic으로 :",info.model_dump())
print("info를 dic으로 :",info.__dict__)

info를 json으로 :  {"capital":"Seoul","population":51,"language":"Korean","currency":"South Korean won"}
info를 dic으로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'South Korean won'}
info를 dic으로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'South Korean won'}


In [64]:
type(info.model_dump_json()),type(info.model_dump())

(str, dict)

# 4. LCEL을 활용한 렝체인 생성하기

## 1) 문자열 출력 파서 사용
- invoke :Runnable에 있는 
- StrOutputParser, ChatOllama, PromptTemplate 등은 모두 Runnable로 부터 상속 받음.

In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Return the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## <span style="color:red">2) LCEL을 사용한 간단한 체인 구성</span>

- 파이프연산자(|) 이용

In [66]:
# 프롬프트 템플릿 --> llm --> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

## 3) 복합체인 구성

- 여러 단계의 추론이 필요한 경우(체인 연결)

In [68]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following informat:
    {information}
    Return the name of the country only""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "This country is very famous for its wine"})))

'Italy'

In [69]:
# 나라명 추출 체인 생성
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information":"This country is very famous for its wine"})

'Italy'

### ※복합체인 : 
```
나라설명 - >나라명(country_chain)
                     나라명 -> 수도(capital_chain)
```

In [71]:
final_chain = country_chain | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [72]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain'
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | {"country":country_chain} | capital_chain

In [74]:
final_chain.invoke("This country is very famous for its wine")

'Rome'